In [1]:
fpath = 'README.md'
with open(fpath, 'rb') as f:
    text = f.read()
text = text.decode("utf-8") 

In [19]:
sections = []
section = []
for i, line in enumerate(text.split('\n')):
    if line.startswith('# '):
        if section:
            sections.append(section)
        section = []
    #section.append(line[2:])
    section.append(line)

entries = []
for i, sec in enumerate(sections):
    if i < 1:
        continue
    entry = {}
    #entry['heading'], entry['rest'] = sec[0], sec[1:]
    entry['heading'], entry['rest'] = sec[0][2:].strip(), sec[1:]
    entries.append(entry)
    
from collections import defaultdict

subheading = ""
for entry in entries:
    subheadings = defaultdict(list)
    for line in entry['rest']:
        if line.startswith('* '):
            subheading = line.strip()[2:]
            continue
        if subheading:
            line = line.strip()
            if line:
                subheadings[subheading].append(line)
    entry.pop('rest')
    if subheadings:
        entry['subheadings'] = subheadings
        
errors = []
for entry in entries:
    if 'subheadings' not in entry:
        continue # actually I want to remove these, but deal with it later
    for topic, items in entry['subheadings'].items():
        recs = []
        for item in items:
            #if len(item.split('-')) != 3:
            #    print((topic, item))

            rec = {}
            parts = item.split('-')
            rec['year_part'], rec['rest'] = parts[0], parts[1:]
            try:
                rec['year'] = int(rec['year_part'][1:].strip())
                rec.pop('year_part')
            except ValueError:
                errors.append(item)

            parts = '-'.join(rec['rest']).split('](')
            if len(parts) <2:
                continue
            if len(parts) > 2:
                errors.append(item)
                continue
            rec['title_part'], rec['rest'] = parts
            
            parts = rec['rest'].split(') -') 
            if len(parts) <2:
                continue
            if len(parts) > 2:
                errors.append(item)
                continue
            rec['url'], rec['authors_part'] = parts
            rec.pop('rest')
            
            if rec['url'].endswith('.pdf'):
                rec['is_pdf'] = 'True'
                
            rec['authors'] = rec['authors_part'].strip().split(',')
            rec.pop('authors_part')
            
            recs.append(rec)
            
        if recs:
            entry['subheadings'][topic] = recs

In [20]:
for entry in entries:
    if 'subheadings' not in entry: # yeesh...
        continue
    for recs in entry['subheadings'].values():
        for rec in recs:
            if not isinstance(rec, dict):
                print(rec)
                continue
            if not rec['title_part'].startswith(' ["'):
                print(rec)

{'year': 1997, 'title_part': ' (AdaBoost) ["A Decision-Theoretic Generalization of On-Line Learning and an Application to Boosting"', 'url': 'https://www.sciencedirect.com/science/article/pii/S002200009791504X', 'authors': ['Yoav Freund', ' Robert E Schapire']}
{'year': 1992, 'title_part': ' [A Training Algorithm for Optimal Margin Classifier', 'url': 'https://www.researchgate.net/publication/2376111_A_Training_Algorithm_for_Optimal_Margin_Classifier', 'authors': ['Bernhard E. Boser', ' Isabelle Guyon', ' Vladimir N. Vapnik']}
{'year': 1986, 'title_part': ' (ID3) ["Induction of Decision Trees"', 'url': 'https://link.springer.com/content/pdf/10.1007/BF00116251.pdf', 'is_pdf': 'True', 'authors': ['J. R. Quinlan']}
{'year': 1984, 'title_part': ' (CART) "Classification and Regression Trees" - in lieu of the book, here\'s a [topic summary from 2011', 'url': 'http://pages.stat.wisc.edu/~loh/treeprogs/guide/wires11.pdf', 'is_pdf': 'True', 'authors': ['Breiman L', ' Friedman JH', ' Olshen RA',

In [22]:
# We're not quite there yet, but the last step will look something like this:
for entry in entries:
    if 'subheadings' not in entry: # yeesh...
        continue
    for subheading, recs in entry['subheadings'].items():
        for rec in recs:
            if not 'url' in rec:
                continue
            path = f"{entry['heading']} / {subheading}"
            print((path, rec))

('"Classic" ML / Lasso/elasticnet', {'year': 1996, 'title_part': ' ["Regression Shrinkage and Selection via the Lasso"', 'url': 'https://statweb.stanford.edu/~tibs/lasso/lasso.pdf', 'is_pdf': 'True', 'authors': ['Robert Tibshirani']})
('"Classic" ML / Lasso/elasticnet', {'year': 2005, 'title_part': ' ["Regularization and variable selection via the elastic net"', 'url': 'https://web.stanford.edu/~hastie/Papers/B67.2%20(2005)%20301-320%20Zou%20&%20Hastie.pdf', 'is_pdf': 'True', 'authors': ['Hui Zou', ' Trevor Hastie']})
('"Classic" ML / Boosting', {'year': 1990, 'title_part': ' ["The Strength of Weak Learnability"', 'url': 'http://rob.schapire.net/papers/strengthofweak.pdf', 'is_pdf': 'True', 'authors': ['Robert E. Schapire']})
('"Classic" ML / Bagging', {'year': 1991, 'title_part': ' ["Bagging Predictors"', 'url': 'https://www.stat.berkeley.edu/~breiman/bagging.pdf', 'is_pdf': 'True', 'authors': ['Leo Breiman']})
('"Classic" ML / random forest', {'year': 2001, 'title_part': ' ["Random F

In [6]:
errors

['* Uh... here there be dragons. Maybe just leave some breadcrumbs here?',
 '* Probably discussed sufficiently in the Adam paper',
 '* see backprop',
 '* See also AlexNet',
 '* see also Alex Graves 2013',
 '* see also knowledge distillation below',
 '* http://karpathy.github.io/2019/04/25/recipe/']